# Submission Code
This is still in development


In [ ]:
sj_train, sj_val = split(sj_data, 1.0)
iq_train, iq_val = split(iq_data, 1.0)


submission = pd.read_csv('../data/submission_format.csv')
sj_test = submission[submission['city']=="sj"]
iq_test = submission[submission['city']=="iq"]

def fit_regression(data_train, lag, pred_len):
    linear_regressor = LinearRegression()  # create object for the class

    X, Y = treat_features(data_train, lag=lag)

    linear_regressor.fit(X, Y)  # perform linear regression
    Y_pred = linear_regressor.predict(X)  # make predictions
    error = mean_absolute_error(Y, Y_pred)
    train_mae = error
#     print("Train MAE = {}".format(error))

    X = [y[0] for y in Y[-lag:]]
    Y=[]
    for i in range(0, pred_len):
        nexty = linear_regressor.predict([X])[0][0]
        Y.append(nexty)
        X = X[1:] + [nexty]

    Y = np.exp(Y)-1
    return Y


y = fit_regression(sj_train, 200, len(sj_test))
sj_test['total_cases'] = y.astype(int)
print(sj_test)


In [ ]:
history = iq_train['total_cases']
mod = sm.tsa.statespace.SARIMAX(history, order=(1, 1, 1), seasonal_order=(1, 1, 1, 52),
                                    enforce_stationarity=False, enforce_invertibility=False)

results = mod.fit()
print('AIC:{}'.format(results.aic))

In [ ]:
history = sj_train['total_cases']
history = np.log(1+history)
mod = sm.tsa.statespace.SARIMAX(history, order=(1, 1, 1), seasonal_order=(1, 1, 1, 52),
                                    enforce_stationarity=False, enforce_invertibility=False)
results = mod.fit()
print('AIC:{}'.format(results.aic))
pred_uc = results.get_forecast(steps=len(sj_test))
predictions = pred_uc.predicted_mean.values
print(predictions)
predictions = np.exp(predictions)-1
sj_test['total_cases']= predictions.astype(int)
print(sj_test)

In [ ]:
alll = pd.concat([sj_test, iq_test])
print(alll)
alll.to_csv("submission.csv", index=False)